In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", eos_token="</s>")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel

device = torch.device("cpu")

pretrained_model_config = GPT2Config.from_pretrained("skt/kogpt2-base-v2")
model = GPT2LMHeadModel(pretrained_model_config).to(device)

fine_tuned_model_ckpt = torch.load("/content/drive/MyDrive/11_recipe_generator/model/epoch=3-val_loss=2.19-v1.ckpt", map_location=torch.device("cpu"))
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [ ]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
from tqdm import tqdm

# Perplexity

### ko-wiki

In [ ]:
lines = open("/content/drive/MyDrive/11_recipe_generator/data/processed_wiki_ko.txt", "r", encoding="utf-8").readlines()
test = ''
for i,line in enumerate(lines):
  test += line
  if i == 10000:
    break

In [ ]:
len(test)

33105851

### recipe test-metric set

In [ ]:
lines = open("/content/drive/MyDrive/11_recipe_generator/data/test_metric.txt", "r", encoding="utf-8").readlines()
test = ''
for i,line in enumerate(lines):
  test += line
  if i == 10000:
    break

### perplexity evaluation

In [ ]:
encodings = tokenizer(test, return_tensors='pt')

In [ ]:
import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 256

nlls = []

for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
  begin_loc = max(i + stride - max_length, 0)
  end_loc = min(i + stride, encodings.input_ids.size(1))
  trg_len = end_loc - i
  
  input_ids = encodings.input_ids[:, begin_loc:end_loc].to("cuda")
  target_ids = input_ids.clone()
  target_ids[:, :-trg_len] = -100

  with torch.no_grad():
    outputs = model(input_ids, labels=target_ids)
    neg_log_likelihood = outputs[0] * trg_len
  
  nlls.append(neg_log_likelihood)

ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
print(ppl)

100%|██████████| 1712/1712 [01:42<00:00, 16.72it/s]


tensor(248.3591, device='cuda:0')


# Custom Metric

test for Custom Metric의 형태는 input과 재료 리스트 <br/> input은 `<unused0>요리명<unused1><unused2>재료1$재료2<unused3>`와 같은 형태 <br/>


In [ ]:
# load ingredient dictionary
lines = open("/content/drive/MyDrive/11_recipe_generator/data/ingredient_list.txt")
ingredients_dict = []

for line in lines:
  ingredients_dict.append(line.rstrip())

ingredients_dict = list(set(ingredients_dict))

In [ ]:
import pandas as pd

# 레시피 데이터프레임
testset_for_custom_metric = pd.DataFrame(columns = ['input', 'ingredients'])

lines = open("/content/drive/MyDrive/11_recipe_generator/data/test_for_custom_metric.reg.txt", "r", encoding="utf-8").readlines()

test = ''
for i,line in enumerate(lines):
  inputs = ''
  input_end = line.find('<unused3>')
  inputs = line[:input_end+9]
  
  ingredients = ''
  ingredient_start = line.find('<unused2>')
  ingredient_end = line.find('<unused3>')
  ingredients = line[ingredient_start+9:ingredient_end-1].split('$')

  testset_for_custom_metric.loc[i] = [inputs, ingredients]

In [ ]:
testset_for_custom_metric

,input,ingredients
0,<unused0>김치 비빔 국수<unused1><unused2>신김치$중면$오이$깨...,"[신김치, 중면, 오이, 깨, 고추장, 식초, 올리고당, 간장, 설탕, 맛술, 참기..."
1,<unused0>오버나이트 오트밀<unused1><unused2>오트밀$두유$아몬드...,"[오트밀, 두유, 아몬드, 치아씨드, 시나몬파우더, 꿀, 블루베리, 산딸기, 개인기..."
2,<unused0>템페 구이 & 퀴노아 샐러드 밀프렙<unused1><unused2>...,"[템페, 간장, 식초, 메이플시럽, 올리브오일, 파프리카파우더, 소금, 후추]"
3,<unused0>반려견 케이크<unused1><unused2>계란$쌀가루분말$코코넛...,"[계란, 쌀가루분말, 코코넛오일, 락토프리우유, 꿀, 찐단호박, 두부, 락토프리우유..."
4,<unused0>비건 베이글<unused1><unused2>강력분$통밀가루$소금$황...,"[강력분, 통밀가루, 소금, 황설탕, 드라이이스트, 포도씨오일, 물]"
...,...,...
1738,<unused0>연근유자피클<unused1><unused2>연근$식초$물$물$식초$...,"[연근, 식초, 물, 물, 식초, 유자청, 소금]"
1739,<unused0>브로콜리수프<unused1><unused2>양파$감자$브로콜리$물$...,"[양파, 감자, 브로콜리, 물, 생크림, 파마산치즈가루, 버터, 소금.후추]"
1740,<unused0>블루베리 크로플<unused1><unused2>크루아상생지$블루베리...,"[크루아상생지, 블루베리콤포트, 크림치즈]"
1741,<unused0>가지볶음<unused1><unused2>가지$양파$청양고추$홍고추$...,"[가지, 양파, 청양고추, 홍고추, 대파, 참기름, 통깨, 식용유, 간장, 굴소스,..."


In [ ]:
def inference(prompt, min_length=50, max_length=250, top_p=0.93, top_k=40, repetition_penalty=1.3, no_repeat_ngram_size=0, temperature=0.1):
  input_ids = tokenizer.encode(prompt, return_tensors="pt")
  with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        top_p=float(top_p),
        top_k=int(top_k),
        min_length = int(min_length),
        max_length=int(max_length),
        repetition_penalty = float(repetition_penalty),
        no_repeat_ngram_size = int(no_repeat_ngram_size),
        temperature = float(temperature),
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        use_cache=True
    )
    generated_sentence = tokenizer.decode(generated_ids[0])
  return {
      'result': generated_sentence
  }

In [ ]:
score = 0
df = testset_for_custom_metric.copy()

for i in tqdm(range(len(df))):
  input = df.loc[i]['input']
  ingredient = df.loc[i]['ingredients']

  gen_sentence = inference(input)['result']
  start = gen_sentence.find('<unused4>')
  end = gen_sentence.find('<unused5>')

  gen_sentence = gen_sentence[start+9:end].lstrip()

  temp_score = 0

  # 생성된 레시피에 있는 재료 수 카운트
  a = []
  for ing in ingredients_dict:
    if ing in gen_sentence:
      a.append(ing)
  # 입력한 재료와 일치하는 것 카운트
  intersection = set(a) & set(ingredient) 
  score += len(intersection)/len(a)

print(score / len(df))

100%|██████████| 1743/1743 [2:47:17<00:00,  5.76s/it]

0.24069604821522061


In [ ]:
score = 0
df = testset_for_custom_metric.copy()

for i in tqdm(range(len(df))):
  input = df.loc[i]['input']
  ingredient = df.loc[i]['ingredients']

  gen_sentence = inference(input)['result']
  start = gen_sentence.find('<unused4>')
  end = gen_sentence.find('<unused5>')

  gen_sentence = gen_sentence[start+9:end].lstrip()

  print(gen_sentence)
  temp_score = 0
  cases = len(ingredient)
  for a in ingredient:
    if a in gen_sentence:
      # print(a,'yes')
      temp_score += 1
    else:
      # print(a,'no')
  score += (temp_score / cases)

print(score / len(df))

In [ ]:
print(score / len(df))

0.4666742981427983
